In [2]:
import edward as ed
from edward.models import Normal, Empirical, Gamma
import tensorflow as tf
from tensorflow.contrib.distributions import bijectors
import numpy as np

N = 20
N_test = 100
epochs = 40
layers = [100]  # CONST
activation_fn = "relu"

# Chosen by https://www.random.org/coins/?num=31&cur=60-usd.0025c-ct
r = np.random.RandomState(0b0001101100010011010011110101110)
true_sigma = 3.0
X_train = r.uniform(-4, 4, N)
y_train = X_train ** 3 + r.normal(scale=true_sigma, size=N)
X_train = X_train.reshape([-1, 1])
size_train = N
X_test = np.linspace(-6, 5, N_test)

# BNN in Edward

In [3]:
from edward.models import Normal, Empirical, Gamma

# Multiple options for handling (x, y)...
x = tf.placeholder(tf.float32, [None])
y = tf.placeholder(tf.float32, [None])
gamma = Gamma(6.0, 6.0)
lambd = Gamma(6.0, 6.0)

b1 = Normal(tf.zeros([100]), tf.ones([100]) / tf.sqrt(lambd))
w1 = Normal(tf.zeros([100]), tf.ones([100]) / tf.sqrt(lambd))
a1 = tf.matmul(tf.reshape(x, (-1, 1)), tf.reshape(w1, (1, -1))) + b1
z1 = tf.nn.relu(a1) # [N, 100]
b2 = Normal(tf.zeros([1]), tf.ones([1]) / tf.sqrt(lambd))
w2 = Normal(tf.zeros([100]), tf.ones([100]) / tf.sqrt(lambd))
out = tf.reshape(tf.matmul(z1, tf.reshape(w2, (100, 1))) + b2, [-1])
y_pred = Normal(out, 1.0 / tf.sqrt(gamma))

## HMC in Edward

In [4]:
ITERS = 30000
qb1 = Empirical(tf.Variable(tf.zeros([ITERS, 100])))
qw1 = Empirical(tf.Variable(tf.zeros([ITERS, 100])))
qb2 = Empirical(tf.Variable(tf.zeros([ITERS, 1])))
qw2 = Empirical(tf.Variable(tf.zeros([ITERS, 100])))
qgamma = Empirical(tf.Variable(tf.zeros([ITERS])))
qlambda = Empirical(tf.Variable(tf.zeros([ITERS])))

# Question, how to get HMC to look at all the data at once.....
inference = ed.HMC({w1: qw1, w2: qw2,
                    b1: qb1, b2: qb2,
                    gamma: qgamma, lambd: qlambda},
                   data={y_pred: y})

In [5]:
# Simple version
inference.run(step_size=0.01, n_steps=4)


# Variable-sized mini-batches
# -- Needed for test set??
inference.initialize(n_iter=ITERS, step_size=0.5 / N, n_steps=2)
tf.global_variables_initializer().run()
for _ in xrange(ITERS):
    info = inference.update({
            x: X_train.reshape(-1).astype(np.float32),
            y: y_train.reshape(-1).astype(np.float32)})
    inference.print_progress(info)

30000/30000 [100%] ██████████████████████████████ Elapsed: 77s | Acceptance Rate: 0.260


## Prediction in Edward

Supposedly, various methods work, but in my (LIMITED) experience, I disagree.

In [6]:
# Manually remove the burn in
egamma = tf.nn.softplus(qgamma.params).eval()[ITERS/2:]
elambda = tf.nn.softplus(qlambda.params).eval()[ITERS/2:]
eb1 = qb1.params.eval()[ITERS/2:, :]
ew1 = qw1.params.eval()[ITERS/2:, :]
eb2 = qb2.params.eval()[ITERS/2:, :]
ew2 = qw2.params.eval()[ITERS/2:, :]

## `ed.copy`

Edward's recommended way to sample from posterior predictive (but not Evan's)

In [ ]:
y_post = ed.copy(
    y_pred,
    feed_dict={
        w1: ew1, w2: ew2,
        b1: eb1, b2: eb2,
        gamma: egamma, lambd: elambda,
        x: X_test.reshape(-1).astype(np.float32)})
y_post.sample(100000).eval()

However, each sample uses iid draws from each "distribution" (`ew1`, `ew2`, ...), rather than value from same iteration in the MCMC.

Note: this is what we want with variational methods, but not necessarily MCMC.

In [8]:
# Edward has a nifty progress bar included...
pr = ed.Progbar(target=ITERS/2)

for i in xrange(ITERS/2):
    y_post = ed.get_session().run(y_pred,
        feed_dict={w1: ew1[i, :], w2: ew2[i, :],
        b1: eb1[i, :], b2: eb2[i, :],
        gamma: egamma[i], lambd: elambda[i],
        x: X_test.reshape(-1).astype(np.float32)})
    
    # do something with y_post...
    
    pr.update(i + 1, values={'avg y[0]': y_post[0]})

15000/15000 [100%] ██████████████████████████████ Elapsed: 14s | avg y[0]: -90.855


### TF Problems
TensorFlow will make new elements of the graph if you're not careful. Edward isn't helpful here.

In [ ]:
for i in xrange(ITERS/2):
    y_post = ed.get_session().run(y_pred.mean(),    # Change y_pred to y_pred.mean()
        feed_dict={w1: ew1[i, :], w2: ew2[i, :],
        b1: eb1[i, :], b2: eb2[i, :],
        gamma: egamma[i], lambd: elambda[i],
        x: X_test.reshape(-1).astype(np.float32)})
    # do something...

Would show results, but REALLY slow. Stopped after 12k iters, tried to show mean, got a segfault..... in Python........

The problem is that `y_pred.mean()` **apparently** creates a new `Tensor` every time...

# BNN in Stan
The following are in the Stan langauge (slightly out of order)

```
data {
    int<lower=0> N;
    real x[N];
    real y[N];

    int<lower=0> N_test;
    real x_test[N_test];
}
```

```
parameters {
    real<lower=0> lambda;
    real<lower=0> gamma;
    row_vector[100] b1;
    matrix[1, 100] w1;
    row_vector[1] b2;
    matrix[100, 1] w2;
}

// Easy to interpret prior and likelihood.
model {
    lambda ~ gamma(6.0, 6.0);
    gamma ~ gamma(6.0, 6.0);
    b1 ~ normal(0.0, 1 / sqrt(lambda));
    // Distributions will "vectorize" but not "matricize"
    to_vector(w1) ~ normal(0.0, 1 / sqrt(lambda));
    b2 ~ normal(0.0, 1 / sqrt(lambda));
    to_vector(w2) ~ normal(0.0, 1 / sqrt(lambda));

    y ~ normal(to_vector(network(to_matrix(x, N, 1), w1, b1, w2, b2)), 1 / sqrt(gamma));
}
```

```
functions {
  matrix network(matrix x, matrix w1, row_vector b1, matrix w2, row_vector b2) {
    {
      matrix[rows(x), 100] z;
      z = x * w1 + rep_matrix(b1, rows(x));
      // Theoretically, traversing in column-order is faster
      for (j in 1:100) {
        for (i in 1:rows(x)) {
          // Will not vectorize or matricize
          z[i, j] = fmax(z[i, j], 0.0);
        }
      }
      return z * w2 + rep_matrix(b2, rows(x));
    }
  }
}

// data, transformed data, parameters, transformed parameters, model

generated quantities {
  matrix[N_test, 1] y_test;
  {
    matrix[N_test, 1] out;
    out = network(to_matrix(x_test, N_test, 1), w1, b1, w2, b2);
    for (i in 1:N_test) {
      y_test[i, 1] = normal_rng(out[i, 1], 1 / sqrt(gamma));
    }
  }
}
```

In [ ]:
import pystan

# Compiles code -- can pass in a string instead.
sm = pystan.StanModel(file='toy.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_371be447302174281a07fb2533155549 NOW.


In [ ]:
d = {'x': X_train.reshape(-1),
     'y': y_train,
     'N': len(y_train),
     'x_test': X_test.reshape(-1),
     'N_test': len(X_test)}

# 30000 burn-in, 30000 samples, default NUTS sampler, 2 parallel chains
fit = sm.sampling(data=d, iter=30000, chains=2)

Logging is good, but if using Jupyter, logs to terminal instead (at least by default)

In [ ]:
# Posterior predictive samples
y_sample = fit.extract(permuted=True, inc_warmup=False)['y_test']